<a href="https://colab.research.google.com/github/MarcosWBeltrami/Projeto_Classificacao_Churn/blob/main/projeto_classificacao_churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto Classificação de Churn
### Este projeto tem como objetivo analisar dados de clientes que possam ser possiveis churn para a empresa, e alcançar a melhor forma de conseguir identificar estes clientes.
###### Churn é uma métrica que indica o quanto sua empresa perdeu de receita ou clientes.

# 1 - Imports PySpark e Dados

In [42]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [44]:
spark = SparkSession.builder.getOrCreate()
dados = spark.read.csv('dados_clientes.csv', sep=',', header=True, inferSchema=True)

# 1.1 - Forma que os dados foram recebidos

In [45]:
dados.show()

+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+
| id|Churn|Mais65anos|Conjuge|Dependentes|MesesDeContrato|TelefoneFixo|MaisDeUmaLinhaTelefonica|   Internet|   SegurancaOnline|      BackupOnline| SeguroDispositivo|    SuporteTecnico|           TVaCabo|   StreamingFilmes|TipoContrato|ContaCorreio| MetodoPagamento|MesesCobrados|
+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+
|  0|  Nao|         0|    Sim|        Nao|              1|         Nao|    SemServicoTelefonico|        DSL|               Nao|               Sim|              

# 2 -Tranformação em Binarios
#### Identificar quais colunas são possiveis fazer a tranformação e realizar a correção dos dados para melhor analise dos dados.

In [46]:
colunasBinarias = [
    'Churn',
    'Conjuge',
    'Dependentes',
    'TelefoneFixo',
    'MaisDeUmaLinhaTelefonica',
    'SegurancaOnline',
    'BackupOnline',
    'SeguroDispositivo',
    'SuporteTecnico',
    'TVaCabo',
    'StreamingFilmes',
    'ContaCorreio'
]

In [47]:
Tcolunas = [f.when(f.col(c)=='Sim', 1).otherwise(0).alias(c) for c in colunasBinarias]


for coluna in reversed(dados.columns):
  if coluna not in colunasBinarias:
    Tcolunas.insert(0, coluna)


dataset = dados.select(Tcolunas)


internet = dataset.groupBy('id').pivot('Internet').agg(f.lit(1)).na.fill(0)
tipoContrato = dataset.groupBy('id').pivot('TipoContrato').agg(f.lit(1)).na.fill(0)
metodoPagamento = dataset.groupBy('id').pivot('MetodoPagamento').agg(f.lit(1)).na.fill(0)

# 3 - Join e Renomeação

In [48]:
dataset = dataset\
    .join(internet, 'id', how='inner')\
    .join(tipoContrato, 'id', how='inner')\
    .join(metodoPagamento, 'id', how='inner')\
    .select(
        '*',
        f.col('DSL').alias('Internet_DSL'), 
        f.col('FibraOptica').alias('Internet_FibraOptica'), 
        f.col('Nao').alias('Internet_Nao'), 
        f.col('Mensalmente').alias('TipoContrato_Mensalmente'), 
        f.col('UmAno').alias('TipoContrato_UmAno'), 
        f.col('DoisAnos').alias('TipoContrato_DoisAnos'), 
        f.col('DebitoEmConta').alias('MetodoPagamento_DebitoEmConta'), 
        f.col('CartaoCredito').alias('MetodoPagamento_CartaoCredito'), 
        f.col('BoletoEletronico').alias('MetodoPagamento_BoletoEletronico'), 
        f.col('Boleto').alias('MetodoPagamento_Boleto')        
    ).drop(
        'Internet', 'TipoContrato', 'MetodoPagamento', 'DSL', 
        'FibraOptica', 'Nao', 'Mensalmente', 'UmAno', 'DoisAnos', 
        'DebitoEmConta', 'CartaoCredito', 'BoletoEletronico', 'Boleto'
    )

# 4 - Tratamento das Colunas

In [49]:
dataset = dataset.withColumnRenamed('Churn', 'label')


x = dataset.columns
x.remove('label')
x.remove('id')


assembler = VectorAssembler(inputCols=x, outputCol='features')
dataset_pred = assembler.transform(dataset).select('features', 'label')

In [50]:
Seed = 101
treino, teste = dataset_pred.randomSplit([0.7, 0.3], seed=Seed)


lr = LogisticRegression()


modelo_lr = lr.fit(treino)
previsoes_lr_teste = modelo_lr.transform(teste)


resumo_lr_treino = modelo_lr.summary
resumo_lr_treino.accuracy

0.7849014709963918

# 5 - Funções

In [51]:
def resultado_teste(teste):
  print('Decision Tree Classifier - Tuning')
  print("="*40)
  print("Dados de Teste")
  print("="*40)
  print("Matriz de Confusão")
  print("-"*40)
  calcula_mostra_matriz_confusao(previsoes_dtc_cv_teste, normalize=False)
  print("-"*40)
  print("Métricas")
  print("-"*40)
  print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "accuracy"}))
  print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
  print("Recall: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
  print("F1: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

In [52]:
def resultado_treino_teste(treino, teste):
  print('Decision Tree Classifier')
  print("="*40)
  print("Dados de Treino")
  print("="*40)
  print("Matriz de Confusão")
  print("-"*40)
  calcula_mostra_matriz_confusao(treino, normalize=False)
  print("-"*40)
  print("Métricas")
  print("-"*40)
  print("Acurácia: %f" % evaluator.evaluate(treino, {evaluator.metricName: "accuracy"}))
  print("Precisão: %f" % evaluator.evaluate(treino, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
  print("Recall: %f" % evaluator.evaluate(treino, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
  print("F1: %f" % evaluator.evaluate(treino, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))
  print("")
  print("="*40)
  print("Dados de Teste")
  print("="*40)
  print("Matriz de Confusão")
  print("-"*40)
  calcula_mostra_matriz_confusao(teste, normalize=False)
  print("-"*40)
  print("Métricas")
  print("-"*40)
  print("Acurácia: %f" % evaluator.evaluate(teste, {evaluator.metricName: "accuracy"}))
  print("Precisão: %f" % evaluator.evaluate(teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
  print("Recall: %f" % evaluator.evaluate(teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
  print("F1: %f" % evaluator.evaluate(teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

In [53]:
def calcula_mostra_matriz_confusao(df_transform_modelo, normalize=False, percentage=True):
  tp = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 1)).count()
  tn = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 0) & (f.col('prediction') == 0)).count()
  fp = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 0) & (f.col('prediction') == 1)).count()
  fn = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 0)).count()
  
  valorP = 1
  valorN = 1

  if normalize:
    valorP = tp + fn
    valorN = fp + tn
  
  if percentage and normalize:
    valorP = valorP / 100
    valorN = valorN / 100

  print(' '*20, 'Previsto')
  print(' '*15, 'Churn', ' '*5 ,'Não-Churn')
  print(' '*4, 'Churn', ' '*6, int(tp/valorP), ' '*7, int(fn/valorP))
  print('Real')
  print(' '*4, 'Não-Churn', ' '*2, int(fp/valorN), ' '*7, int(tn/valorN))

# 6.1 - Tratamento formato DecisionTreeClassifier (Treino e Teste)




In [54]:
dtc = DecisionTreeClassifier(seed = Seed)
modelo_dtc = dtc.fit(treino)
previsoes_dtc_treino = modelo_dtc.transform(treino)


evaluator = MulticlassClassificationEvaluator()
evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: 'accuracy'})


previsoes_dtc_teste = modelo_dtc.transform(teste)
evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: 'accuracy'})

0.7714831317632082

# 6.1.1 - Resultado dos Testes

In [55]:
resultado_treino_teste(previsoes_dtc_treino, previsoes_dtc_teste)

Decision Tree Classifier
Dados de Treino
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        2784         827
Real
     Não-Churn    674         2921
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.791701
Precisão: 0.805090
Recall: 0.770978
F1: 0.787664

Dados de Teste
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        1181         382
Real
     Não-Churn    336         1243
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.771483
Precisão: 0.778510
Recall: 0.755598
F1: 0.766883


# 6.2 - Tratamento formato DecisionTreeClassifier (Treino)

In [56]:
dtc = DecisionTreeClassifier(seed = Seed)


grid = ParamGridBuilder()\
        .addGrid(dtc.maxDepth,[2, 5, 10])\
        .addGrid(dtc.maxBins,[10, 32, 45])\
        .build()

In [57]:
evaluator = MulticlassClassificationEvaluator()


dtc_cv = CrossValidator(
    estimator=dtc,
    estimatorParamMaps=grid,
    evaluator=evaluator,
    numFolds=3,
    seed=Seed
)


modelo_dtc_cv = dtc_cv.fit(treino)


previsoes_dtc_cv_teste = modelo_dtc_cv.transform(teste)

# 6.2.1 - Resultado dos Testes

In [58]:
resultado_teste(previsoes_dtc_cv_teste)

Decision Tree Classifier - Tuning
Dados de Teste
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        1327         236
Real
     Não-Churn    423         1156
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.790261
Precisão: 0.758286
Recall: 0.849008
F1: 0.801087


# 7.1 - Tratamento formato RandomForestClassifier (Treino e Teste)

In [59]:
rfc = RandomForestClassifier(seed = Seed)


modelo_rfc = rfc.fit(treino)
previsoes_rfc_treino = modelo_rfc.transform(treino)


previsoes_rfc_teste = modelo_rfc.transform(teste)

#7.1.1 - Resultado dos Testes

In [60]:
resultado_treino_teste(previsoes_rfc_treino, previsoes_rfc_teste)

Decision Tree Classifier
Dados de Treino
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        2950         661
Real
     Não-Churn    884         2711
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.785595
Precisão: 0.769431
Recall: 0.816948
F1: 0.792478

Dados de Teste
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        1257         306
Real
     Não-Churn    416         1163
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.770210
Precisão: 0.751345
Recall: 0.804223
F1: 0.776885


# 7.2 - Tratamento formato RandomForestClassifier (Treino)



In [61]:
rfc = RandomForestClassifier(seed = Seed)


grid = ParamGridBuilder()\
        .addGrid(rfc.maxDepth,[2, 5, 10])\
        .addGrid(rfc.maxBins,[10, 32, 45])\
        .addGrid(rfc.numTrees,[10, 20, 50])\
        .build()

In [62]:
evaluator = MulticlassClassificationEvaluator()


rfc_cv = CrossValidator(
    estimator=rfc,
    estimatorParamMaps=grid,
    evaluator=evaluator,
    numFolds=3,
    seed=Seed
)


modelo_rfc_cv = rfc_cv.fit(treino)


previsoes_rfc_cv_teste = modelo_rfc_cv.transform(teste)

#7.2.1 - Resultado dos Testes



In [63]:
resultado_teste(previsoes_rfc_cv_teste)

Decision Tree Classifier - Tuning
Dados de Teste
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        1327         236
Real
     Não-Churn    423         1156
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.790261
Precisão: 0.758286
Recall: 0.849008
F1: 0.801087


#8 - Modelo Definido
#### Com base nos analises anteriores foi definido como esta a melhor forma de encontrar o churn

In [64]:
melhor_modelo_rfc_cv = modelo_rfc_cv.bestModel


rfc_tunning = RandomForestClassifier(maxDepth=10, maxBins=45, numTrees=10, seed=Seed)
modelo_rfc_tunning = rfc_tunning.fit(dataset_pred)

# 8.1 - Definição por um cliente do Zero
#### Foi criado este cliente para demonstrar que com base somente nos dados necessarios é possivel fazer o levantamento.

In [65]:
novo_cliente = [{
    'Mais65anos': 0,
    'MesesDeContrato': 1,
    'MesesCobrados': 45.30540797610398,
    'Conjuge': 0,
    'Dependentes': 0,
    'TelefoneFixo': 0,
    'MaisDeUmaLinhaTelefonica': 0,
    'SegurancaOnline': 0,
    'BackupOnline': 0,
    'SeguroDispositivo': 0,
    'SuporteTecnico': 0,
    'TVaCabo': 1,
    'StreamingFilmes': 1,
    'ContaCorreio': 1,
    'Internet_DSL': 1,
    'Internet_FibraOptica': 0,
    'Internet_Nao': 0,
    'TipoContrato_Mensalmente': 1,
    'TipoContrato_UmAno': 0,
    'TipoContrato_DoisAnos': 0,
    'MetodoPagamento_DebitoEmConta': 0,
    'MetodoPagamento_CartaoCredito': 0,
    'MetodoPagamento_BoletoEletronico': 1,
    'MetodoPagamento_Boleto': 0
}]

#8.2 Tratamento do novo cliente

In [66]:
novo_cliente = spark.createDataFrame(novo_cliente)
novo_cliente.show()

+------------+-------+------------+-----------+------------+--------------------+------------+----------+------------------------+-----------------+---------------+----------------------+--------------------------------+-----------------------------+-----------------------------+---------------+-----------------+---------------+--------------+-------+------------+---------------------+------------------------+------------------+
|BackupOnline|Conjuge|ContaCorreio|Dependentes|Internet_DSL|Internet_FibraOptica|Internet_Nao|Mais65anos|MaisDeUmaLinhaTelefonica|    MesesCobrados|MesesDeContrato|MetodoPagamento_Boleto|MetodoPagamento_BoletoEletronico|MetodoPagamento_CartaoCredito|MetodoPagamento_DebitoEmConta|SegurancaOnline|SeguroDispositivo|StreamingFilmes|SuporteTecnico|TVaCabo|TelefoneFixo|TipoContrato_DoisAnos|TipoContrato_Mensalmente|TipoContrato_UmAno|
+------------+-------+------------+-----------+------------+--------------------+------------+----------+------------------------+----

In [67]:
assembler = VectorAssembler(inputCols=x, outputCol= 'features')
novo_cliente_pred = assembler.transform(novo_cliente).select('features')
novo_cliente_pred.show(truncate=False)

+----------------------------------------------------------------------------+
|features                                                                    |
+----------------------------------------------------------------------------+
|(24,[1,2,11,12,13,14,17,22],[1.0,45.30540797610398,1.0,1.0,1.0,1.0,1.0,1.0])|
+----------------------------------------------------------------------------+



# 9 - Resultado Final
#### Com base nos dados recebidos foi possivel fazer a analise se sera um churn ou não, como é possivel ver na coluna (prediction), sendo assim concluido o projeto.

In [68]:
modelo_rfc_tunning.transform(novo_cliente_pred).show()

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|(24,[1,2,11,12,13...|[1.24640781461703...|[0.12464078146170...|       1.0|
+--------------------+--------------------+--------------------+----------+

